In [0]:
#Read NPI extracts from the bronze layer
df=spark.read.format("parquet").load("/mnt/bronze/npi_extract/")

df.createOrReplaceTempView("staging_npi_extract")

In [0]:
%sql
SELECT * FROM staging_npi_extract

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.npi_extract(
  npi_id STRING,
  first_name STRING,
  last_name STRING,
  last_updated STRING,
  organisation_name STRING,
  position STRING,
  inserted_date DATE,
  updated_date DATE,
  is_current_flag BOOLEAN
)

In [0]:
%sql
MERGE INTO silver.npi_extract AS target
USING staging_npi_extract AS source
ON target.npi_id = source.npi_id AND target.is_current_flag = true
WHEN MATCHED AND
  target.first_name != source.first_name OR
  target.last_name != source.last_name OR
  target.last_updated != source.last_updated OR
  target.organisation_name != source.organisation_name OR
  target.position != source.position
THEN UPDATE SET 
  target.is_current_flag = false,
  target.updated_date = current_date

WHEN NOT MATCHED
THEN INSERT (
  npi_id,
  first_name,
  last_name,
  last_updated,
  organisation_name,
  position,
  inserted_date,
  updated_date,
  is_current_flag
)
VALUES (
  source.npi_id,
  source.first_name,
  source.last_name,
  source.last_updated,
  source.organisation_name,
  source.position,
  current_timestamp(),
  current_timestamp(),
  true
)